## Preliminary Setup

In [1]:
!pip install pdfplumber tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 47.1 MB/s eta 0:00:00


In [2]:
import re
from pathlib import Path
import pickle
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pdfplumber

url = 'https://drive.google.com/uc?export=download&id=1jKoXzzd5bqpcJeT_TtchmoGza3WRY8Xc'
output = 'Marine_Revenue_FY20-FY24.pdf'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1jKoXzzd5bqpcJeT_TtchmoGza3WRY8Xc
To: /content/Marine_Revenue_FY20-FY24.pdf
100%|██████████| 1.64M/1.64M [00:00<00:00, 47.8MB/s]


'Marine_Revenue_FY20-FY24.pdf'

## Summary Table

In [4]:
import re
from pathlib import Path
import pdfplumber
import pandas as pd

# ---- Paths and output ----
PDF = Path("/content/Marine_Revenue_FY20-FY24.pdf")
OUT = PDF.parent / "Marine_Revenue_FY20-FY24_summary_table.csv"

# ---- Columns ----
COLS = ["Country", "Installation", "FY16", "FY17", "FY18", "FY19", "FY20 thru SEP", "Annualized FY20"]

# ---- Normalization maps ----
COUNTRY_MAP = {"japan": "Japan", "korea": "Korea"}
INSTALL_MAP = {
    "campfuji": "Camp Fuji", "campschwab": "Camp Schwab", "camphansen": "Camp Hansen",
    "campcourtney": "Camp Courtney", "campbutler/foster": "Camp Butler/Foster",
    "campfutenma": "Camp Futenma", "campkinser": "Camp Kinser",
    "iwakuni": "Iwakuni", "campmujuk": "Camp Mujuk"
}

TOTAL_RX = re.compile(r"^totalrevenue$", re.IGNORECASE)

def compact(s):
    return re.sub(r"\s+", "", s or "").strip()

def normalize_country(s):
    key = compact(s).lower()
    return COUNTRY_MAP.get(key, (s or "").strip())

def normalize_installation(s):
    s2 = re.sub(r"(?<=\w)\s(?=\w)", "", s or "")
    key = compact(s2).lower()
    return INSTALL_MAP.get(key, s2.strip())

def money_to_float(s):
    if not s:
        return None
    s = str(s).replace(",", "").replace("$", "").strip()
    if s.startswith("(") and s.endswith(")"):
        s = "-" + s[1:-1]
    try:
        return float(s)
    except:
        return None

# ---------- Basic line/character grouping ----------
def chars_to_lines(chars, y_tol=3):
    chars = sorted(chars, key=lambda c: (c["top"], c["x0"]))
    lines, cur, last_y = [], [], None
    for ch in chars:
        if last_y is None or abs(ch["top"] - last_y) <= y_tol:
            cur.append(ch)
            last_y = ch["top"] if last_y is None else (last_y + ch["top"]) / 2
        else:
            if cur:
                lines.append(cur)
            cur = [ch]
            last_y = ch["top"]
    if cur:
        lines.append(cur)
    return lines

def _merge_chars(chars):
    txt = "".join(c["text"] for c in chars)
    return {"text": txt, "x0": min(c["x0"] for c in chars), "x1": max(c["x1"] for c in chars)}

def line_to_words(line_chars, gap=3):
    if not line_chars:
        return []
    line_chars = sorted(line_chars, key=lambda c: c["x0"])
    words, cur = [], [line_chars[0]]
    for ch in line_chars[1:]:
        if ch["x0"] - cur[-1]["x1"] <= gap:
            cur.append(ch)
        else:
            words.append(_merge_chars(cur))
            cur = [ch]
    if cur:
        words.append(_merge_chars(cur))
    return words

# ---------- Header detection and column splitting ----------
def detect_cuts_from_header(wline):
    """
    Estimate column split lines based on approximate positions of header texts;
    if not found, fallback to using average step width.
    """
    fy_pattern = re.compile(r"^(fy)?\d+(\.\d+)?$|annualized", re.IGNORECASE)
    boxes = []
    for w in wline:
        key = compact(w["text"]).lower()
        mid = (w["x0"] + w["x1"]) / 2
        width = w["x1"] - w["x0"]
        if key == "countryinstallation":
            # Special handling for merged header
            total_len = len("countryinstallation")
            country_len = len("country")
            proportion = country_len / total_len
            mid1 = w["x0"] + proportion * width * 0.5
            mid2 = w["x1"] - (1 - proportion) * width * 0.5
            boxes.append((mid1, "country"))
            boxes.append((mid2, "installation"))
        elif "country" in key:
            boxes.append((mid, "country"))
        elif "installation" in key:
            boxes.append((mid, "installation"))
        elif fy_pattern.match(key):
            boxes.append((mid, "fy"))
    if len(boxes) < len(COLS):
        left = min(w["x0"] for w in wline)
        right = max(w["x1"] for w in wline)
        step = (right - left) / len(COLS)
        return [left + i * step for i in range(len(COLS) + 1)]
    boxes.sort()
    xs = [x for x, _ in boxes]
    cuts = [-1e9] + [(xs[i] + xs[i + 1]) / 2 for i in range(len(xs) - 1)] + [1e9]
    return cuts

def find_header_index(lines_words):
    """
    Automatically find the header row that contains keywords like Country / Installation / FYxx.
    Return the index of that row; return None if not found.
    """
    for idx, wline in enumerate(lines_words):
        texts = [compact(w.get("text","")).lower() for w in wline]
        joined = " ".join(texts)
        # Must contain both "country" and "installation", and also "fy"
        if ("country" in joined and "installation" in joined and "fy" in joined):
            return idx
    return None

# ---------- Data row splitting and fixing ----------
def assign_row(wline, cuts):
    buckets = [[] for _ in range(len(COLS))]
    for w in sorted(wline, key=lambda d: d["x0"]):
        xm = (w["x0"] + w["x1"]) / 2
        for i in range(len(COLS)):
            if cuts[i] <= xm < cuts[i + 1]:
                buckets[i].append(w["text"])
                break

    def join_txt(t):
        return re.sub(r"(?<=\w)\s(?=\w)", "", " ".join(t)).strip()
    def join_money(t):
        return re.sub(r"[^\d\-\.\$,\(\)]", "", "".join(t).replace(" ", ""))

    row = []
    for i, toks in enumerate(buckets):
        row.append(join_txt(toks) if i <= 1 else join_money(toks))
    return row

def fix_country_installation(row):
    c, i = row[0], row[1]
    if i:
        return [normalize_country(c), normalize_installation(i)] + row[2:]
    # Handle merged cases like "KoreaCampMujuk" or "JapanCampFuji"
    c_compact = compact(c)
    known_countries = ["Korea", "Japan"]
    for country in known_countries:
        if c_compact.startswith(country):
            install_part = c[len(country):]  # Preserve original formatting
            return [normalize_country(country), normalize_installation(install_part)] + row[2:]
    # Fallback to searching for "Camp"
    m = re.search(r"Camp", c, re.IGNORECASE)
    if m:
        pos = m.start()
        country_part = c[:pos]
        install_part = c[pos:]
        return [normalize_country(country_part), normalize_installation(install_part)] + row[2:]
    return [normalize_country(c), normalize_installation(i)] + row[2:]

# ---------- Extract a single page ----------
def extract_page_slot_revenue(page):
    # Extract text as characters → lines → words
    lines_chars = chars_to_lines(page.chars)
    lines_words = [line_to_words(ln) for ln in lines_chars if ln]

    # Find header
    hdr_idx = find_header_index(lines_words)
    if hdr_idx is None:
        return pd.DataFrame(columns=COLS)  # Return empty DataFrame if not found

    hdr = lines_words[hdr_idx]
    cuts = detect_cuts_from_header(hdr)

    rows = []
    for i in range(hdr_idx + 1, len(lines_words)):
        txt = compact("".join(w["text"] for w in lines_words[i]))
        if TOTAL_RX.match(txt):
            break
        row = assign_row(lines_words[i], cuts)
        row = fix_country_installation(row)
        # Keep rows where either Country/Installation or any numeric column has data
        if any(row[2:]) or row[0] or row[1]:
            rows.append(row)

    df = pd.DataFrame(rows, columns=COLS)
    # Clean numeric values
    for c in COLS[2:]:
        df[c] = df[c].apply(lambda x: None if not x else x)
        df[c] = df[c].apply(money_to_float)
    return df

# ---------- Main program: extract pages 1, 35, 115 ----------
def main():
    target_pages_1based = [1, 35, 73, 115, 157]
    target_indices = [p - 1 for p in target_pages_1based]  # Convert to 0-based index

    all_dfs = []
    with pdfplumber.open(PDF) as pdf:
        for i, pidx in enumerate(target_indices):
            if pidx < 0 or pidx >= len(pdf.pages):
                print(f"⚠️ Specified page number out of range: {pidx+1}")
                continue
            page = pdf.pages[pidx]
            df = extract_page_slot_revenue(page)
            if not df.empty:
                df.insert(0, "Page", pidx + 1)  # Preserve actual 1-based page number
                all_dfs.append(df)
            else:
                print(f"⚠️ Page {pidx+1} did not detect Slot Revenue table.")

    if all_dfs:
        out_df = pd.concat(all_dfs, ignore_index=True)
        out_df.to_csv(OUT, index=False, encoding="utf-8-sig")
        print(f"✅ Slot Revenue: {len(out_df)} rows exported → {OUT}")
    else:
        print("⚠️ No table data retrieved from any page.")

if __name__ == "__main__":
    main()


✅ Slot Revenue: 97 rows exported → /content/Marine_Revenue_FY20-FY24_summary_table.csv


## Detail Table

In [7]:
# ---- Path configuration ----
PDF = Path("Marine_Revenue_FY20-FY24.pdf")  # your PDF filename
OUT = PDF.parent / "Marine_Revenue_FY20-FY24_detail.csv"

# ---- Columns and extraction parameters ----
COLS = ["Loc #", "Location", "Month", "Revenue", "NAFI Amt", "Annual Revenue", "Annual NAFI"]

# Baseline tuning parameters. These values work for the majority of pages.
# See SPECIAL_PARAMS below for overrides used on specific page ranges.
BASE_Y_TOL = 3.0        # vertical merge tolerance
BASE_X_JOIN = 4.0       # horizontal character join tolerance
BASE_GAP_RATIO = 0.8    # smart join whitespace threshold
BASE_DROP_MIN = 2       # minimum number of non-empty columns to keep a row
BASE_EDGE_PAD = 15      # extra padding around column cut lines
BASE_LEFT_SHIFT_MONTH = 18   # slight left shift for the Month column
BASE_LEFT_SHIFT_REVENUE = 8  # slight left shift for the Revenue column

# Page ranges with customised parameter values. The keys are range objects
# inclusive of the start and end page numbers. For example, range(163, 167)
# covers pages 163, 164, 165 and 166. Values that are not specified in a
# particular override fall back to the baseline values defined above.
SPECIAL_PARAMS = {
    # Pages 163–166 require slightly larger tolerances and shifts to account
    # for the layout on those pages.
    range(163, 167): {
        "Y_TOL": 6.5,
        "X_JOIN": 2.6,
        "GAP_RATIO": 0.52,
        "DROP_MIN": 1,
        "EDGE_PAD": 8,
        "LEFT_SHIFT_MONTH": 14,
        "LEFT_SHIFT_REVENUE": 5,
    },
    # Pages 172–177 have tighter spacing horizontally but require a larger
    # vertical tolerance; adjust the shifts modestly.
    range(172, 178): {
        "Y_TOL": 4.0,
        "X_JOIN": 4.5,
        "GAP_RATIO": 0.8,
        "DROP_MIN": 2,
        "EDGE_PAD": 15,
        "LEFT_SHIFT_MONTH": 20,
        "LEFT_SHIFT_REVENUE": 10,
    },
    # Pages 178–190 contain very dense tables with wider spacing between rows
    # and larger numbers; increase tolerances and edge padding accordingly.
    range(178, 191): {
        "Y_TOL": 5.0,
        "X_JOIN": 5.0,
        "GAP_RATIO": 0.8,
        "DROP_MIN": 2,
        "EDGE_PAD": 20,
        "LEFT_SHIFT_MONTH": 22,
        "LEFT_SHIFT_REVENUE": 12,
    },
}

# ---- Regular expressions ----
MONTH_FULL = re.compile(r"^(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[-/]?\d{2}$", re.I)
MONTH_REV = re.compile(r"^\d{2}[-/](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)$", re.I)
MONTH_ONLY = re.compile(r"^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)$", re.I)
NUM_RE = re.compile(r"^-?\(?\d{1,3}(?:,\d{3})*(?:\.\d+)?\)?$")
INT_RE = re.compile(r"^\d+$")
YEAR_FRAG = {-17, -18, -19, -20}
NUM_RE_STRICT = re.compile(r"^-?\(?\d{1,3}(?:,\d{3})*(?:\.\d+)?\)?$")
money_re = re.compile(r'^-?\(?\d+(?:\.\d+)?\)?$')

# ---- Header patterns ----
HEADER_PATTERNS = {
    "Loc #": re.compile(r"^Loc\s*#?$", re.I),
    "Location": re.compile(r"^Location$", re.I),
    "Month": re.compile(r"^Month$", re.I),
    "Revenue": re.compile(r"^Revenue$", re.I),
    "NAFI Amt": re.compile(r"^NAFI\s*Amt$", re.I),
    "Annual Revenue": re.compile(r"^Annual\s+Revenue$", re.I),
    "Annual NAFI": re.compile(r"^Annual\s+NAFI$", re.I),
}
HEADER_ORDER = ["Loc #", "Location", "Month", "Revenue", "NAFI Amt", "Annual Revenue", "Annual NAFI"]

# ==== Noise keywords (exclude tokens not part of data rows) ====
EXCLUDE_LINE_PATTERNS = [
    re.compile(r"^ARMP\s+Marine\s+Slot\s+Report$", re.I),
    re.compile(r"^Monthly\s+Summary\s+by\s+Location$", re.I),
    re.compile(r"^Slot\s+Revenue$", re.I),
    re.compile(r"^NAFI\s+Reimbursement\s+from\s+ARMP$", re.I),
    re.compile(r"^Region$", re.I),
    re.compile(r"^Loc\s*#?$", re.I),
    re.compile(r"^Location$", re.I),
    re.compile(r"^Month$", re.I),
    re.compile(r"^Revenue$", re.I),
    re.compile(r"^NAFI\s*Amt$", re.I),
    re.compile(r"^Annual\s+Revenue$", re.I),
    re.compile(r"^Annual\s+NAFI$", re.I),
]

def _is_exclude_token(t):
    txt = t["text"].strip()
    return any(p.match(txt) for p in EXCLUDE_LINE_PATTERNS)

# ---------------- Basic helpers ----------------
def chars_to_words(page, x_tol, y_tol):
    """Group characters from a pdfplumber page into word tokens.
    Parameters ``x_tol`` and ``y_tol`` control the tolerance for horizontal
    character joins and vertical line grouping, respectively. These values
    will be supplied from the current page's parameter set.
    """
    chars = sorted(page.chars, key=lambda c: (round(c["top"], 1), c["x0"]))
    lines, words = [], []
    for ch in chars:
        if not lines or abs(ch["top"] - lines[-1]["y"]) > y_tol:
            lines.append({"y": ch["top"], "chars": [ch]})
        else:
            lines[-1]["chars"].append(ch)
    for line in lines:
        row = sorted(line["chars"], key=lambda c: c["x0"])
        cur = [row[0]]
        for c in row[1:]:
            if c["x0"] - cur[-1]["x1"] <= x_tol:
                cur.append(c)
            else:
                words.append({
                    "text": "".join(x["text"] for x in cur),
                    "x0": cur[0]["x0"],
                    "x1": cur[-1]["x1"],
                    "top": line["y"],
                })
                cur = [c]
        if cur:
            words.append({
                "text": "".join(x["text"] for x in cur),
                "x0": cur[0]["x0"],
                "x1": cur[-1]["x1"],
                "top": line["y"],
            })
    return words

def smart_join(tokens, x_tol, gap_ratio):
    """Join tokens horizontally, inserting spaces when appropriate.
    The ``gap_ratio`` parameter determines how aggressive the join is when
    encountering whitespace between tokens. Higher values will insert
    spaces more frequently.
    """
    if not tokens:
        return ""
    tokens = sorted(tokens, key=lambda w: w["x0"])
    s = tokens[0]["text"]
    for i in range(1, len(tokens)):
        prev = tokens[i - 1]
        curr = tokens[i]
        gap = curr["x0"] - prev["x1"]
        prev_txt = prev["text"].strip()
        curr_txt = curr["text"].strip()
        # handle commas and numbers specially
        if (
            gap <= x_tol or
            (prev_txt.isdigit() and curr_txt == ",") or
            (prev_txt == "," and curr_txt.isdigit()) or
            (prev_txt.isdigit() and curr_txt.isdigit()) or
            ("." in prev_txt and curr_txt.isdigit())
        ):
            s += curr["text"]
        else:
            s += " " + curr["text"]
    return s.strip()

def assign_bin(xmid, cuts, edge_pad):
    """Return the index of the bucket into which the x-coordinate ``xmid`` falls.
    ``edge_pad`` expands each cut line slightly on either side to catch
    borderline tokens.
    """
    for i in range(len(cuts) - 1):
        if cuts[i] - edge_pad <= xmid <= cuts[i + 1] + edge_pad:
            return i
    return None

def header_y_of(words):
    ys = [w["top"] for w in words if w["text"] in ("Location", "Month", "Revenue")]
    return min(ys) if ys else 0

def group_by_y(words, y_tol):
    lines = []
    for w in sorted(words, key=lambda w: (round(w["top"], 1), w["x0"])):
        if not lines or abs(w["top"] - lines[-1]["y"]) > y_tol:
            lines.append({"y": w["top"], "tokens": [w]})
        else:
            lines[-1]["tokens"].append(w)
    return lines

# ---------------- Detect header and cuts ----------------
def detect_header_line(words):
    candidates = []
    for line in group_by_y(words, BASE_Y_TOL):
        texts = [t["text"].strip() for t in line["tokens"]]
        hit = sum(1 for pat in HEADER_PATTERNS.values() if any(pat.match(tx) for tx in texts))
        if hit >= 3:
            candidates.append((line["y"], line["tokens"], hit))
    if not candidates:
        return 0, None
    candidates.sort(key=lambda x: (x[2], x[0]))
    y, toks, _ = candidates[-1]
    return y, toks

def cuts_from_header(words, base_cuts, prev_cuts, left_shift_month, left_shift_revenue):
    """
    Determine each column's cut lines (cuts) based on detected header tokens.
    Return (cuts, header_y). If detection fails for any reason, return
    (prev_cuts or base_cuts, an estimated header_y).
    """
    # First try: detect the header line by finding a line containing ≥3 header titles
    header_y, header_tokens = detect_header_line(words)

    # Provide a fallback header_y (use the minimum y among common columns)
    fallback_hy = header_y_of(words)

    # If a header line is found, derive cuts directly from that line's token x0 values
    if header_tokens:
        # Map each token text on that line to a header field name
        name_to_x0 = {}
        for t in header_tokens:
            tx = t["text"].strip()
            for name, pat in HEADER_PATTERNS.items():
                if pat.match(tx):
                    # Use only the leftmost x0 for that field
                    name_to_x0[name] = min(name_to_x0.get(name, t["x0"]), t["x0"])
        if name_to_x0:
            # Generate each column's left boundary (starts) according to HEADER_ORDER
            # If a field is missing, use the previous page's or base position as an interpolation
            ref = prev_cuts or base_cuts
            starts = []
            for name in HEADER_ORDER:
                if name in name_to_x0:
                    starts.append(float(name_to_x0[name]))
                else:
                    # Estimate a reasonable position using the previous/base column boundaries
                    col_idx = HEADER_ORDER.index(name)
                    left_edge = ref[col_idx]
                    right_edge = ref[col_idx + 1]
                    starts.append((left_edge * 0.65 + right_edge * 0.35))
            # Produce cuts from starts: use the midpoint between adjacent column left edges
            cuts = [-1e9]
            for i in range(1, len(starts)):
                cuts.append((starts[i - 1] + starts[i]) / 2.0)
            cuts.append(1e9)
            # Slightly adjust the Month/Revenue boundaries (consistent with the original left-shift)
            idx_month = HEADER_ORDER.index("Month")
            idx_revenue = HEADER_ORDER.index("Revenue")
            # Note: cuts has one fewer index than columns; the boundary to the right of Month is cuts[idx_month+1]
            # The original logic shifts the boundary left after generating starts; keep that simple behavior here:
            cuts[idx_month + 1] -= left_shift_month
            cuts[idx_revenue + 1] -= left_shift_revenue
            return cuts, (header_y or fallback_hy)
        # Header line found but no field matched → fall back
        return (prev_cuts or base_cuts), (header_y or fallback_hy)

    # ---- No header line found: traverse the page and use the first x0 where each header appears as reference ----
    name_to_x0 = {}
    for name, pat in HEADER_PATTERNS.items():
        xs = [w["x0"] for w in words if pat.match(w["text"].strip())]
        if xs:
            name_to_x0[name] = float(min(xs))
    if name_to_x0:
        ref = prev_cuts or base_cuts
        starts = []
        for name in HEADER_ORDER:
            if name in name_to_x0:
                starts.append(name_to_x0[name])
            else:
                # If missing, interpolate using previous/base boundaries
                col_idx = HEADER_ORDER.index(name)
                left_edge = ref[col_idx]
                right_edge = ref[col_idx + 1]
                starts.append((left_edge * 0.65 + right_edge * 0.35))
        cuts = [-1e9]
        for i in range(1, len(starts)):
            cuts.append((starts[i - 1] + starts[i]) / 2.0)
        cuts.append(1e9)
        idx_month = HEADER_ORDER.index("Month")
        idx_revenue = HEADER_ORDER.index("Revenue")
        cuts[idx_month + 1] -= left_shift_month
        cuts[idx_revenue + 1] -= left_shift_revenue
        return cuts, fallback_hy

    # ---- Still nothing found → final fallback ----
    return (prev_cuts or base_cuts), fallback_hy

def _is_money_token(s: str) -> bool:
    """Return True if a string looks like a numeric or currency value."""
    if s is None:
        return False
    s = str(s).strip().replace(',', '').replace('$', '')
    return bool(re.match(r"^-?\(?\d+(\.\d+)?\)?$", s))


# ---------------- Type-aware refinement of cuts ----------------
def _is_month_token(s: str) -> bool:
    if s is None:
        return False
    s = str(s).strip()
    return bool(
        MONTH_FULL.match(s) or
        MONTH_REV.match(s) or
        MONTH_ONLY.match(s)
    )

def refine_cuts_typeaware(page, cuts, hy):
    words = chars_to_words(page, x_tol=BASE_X_JOIN, y_tol=BASE_Y_TOL)
    body = [w for w in words if w["top"] > hy + 1 and not _is_exclude_token(w)]

    rows, cur_y, cur = [], None, []
    for w in sorted(body, key=lambda w: (round(w["top"], 1), w["x0"])):
        y = round(w["top"], 1)
        if cur_y is None or abs(y - cur_y) <= BASE_Y_TOL:
            cur.append(w)
            cur_y = y if cur_y is None else (cur_y + y) / 2
        else:
            rows.append(cur)
            cur = [w]
            cur_y = y
    if cur:
        rows.append(cur)

    month_right, revenue_left = [], []
    revenue_right, nafi_left = [], []
    nafi_right, annual_rev_left = [], []
    annual_rev_right, annual_nafi_left = [], []

    for r in rows:
        for w in r:
            xmid = (w["x0"] + w["x1"]) / 2
            bi = assign_bin(xmid, cuts, BASE_EDGE_PAD)
            if bi is None:
                continue
            txt = w["text"]
            if bi == 2 and _is_month_token(txt):  # Month bucket
                month_right.append(w["x1"])
            if bi == 3 and _is_money_token(txt):  # Revenue bucket
                revenue_left.append(w["x0"])
                revenue_right.append(w["x1"])
            if bi == 4 and _is_money_token(txt):  # NAFI Amt bucket
                nafi_left.append(w["x0"])
                nafi_right.append(w["x1"])
            if bi == 5 and _is_money_token(txt):  # Annual Revenue bucket
                annual_rev_left.append(w["x0"])
                annual_rev_right.append(w["x1"])
            if bi == 6 and _is_money_token(txt):  # Annual NAFI bucket
                annual_nafi_left.append(w["x0"])

    def pct(a, p):
        return float(np.percentile(a, p)) if a else None

    # Month↔Revenue boundary (cuts[3])
    left_p = pct(month_right, 100)
    right_p = pct(revenue_left, 0)
    if left_p is not None and right_p is not None and right_p > left_p:
        target = (left_p + right_p) / 2
        base = BASE_CUTS[3]
        delta = CLAMP_DELTA[3]
        cuts[3] = max(base - delta, min(base + delta, target))
    if left_p is not None and right_p is None:
        cuts[3] = max(cuts[3], left_p + 1)

    # Revenue↔NAFI boundary (cuts[4])
    left_p = pct(revenue_right, 100)
    right_p = pct(nafi_left, 0)
    if left_p is not None and right_p is not None and right_p > left_p:
        target = (left_p + right_p) / 2
        base = BASE_CUTS[4]
        delta = CLAMP_DELTA[4]
        cuts[4] = max(base - delta, min(base + delta, target))
    if left_p is not None and right_p is None:
        cuts[4] = max(cuts[4], left_p + 1)

    # NAFI Amt↔Annual Revenue boundary (cuts[5])
    left_p = pct(nafi_right, 100)
    right_p = pct(annual_rev_left, 0)
    if left_p is not None and right_p is not None and right_p > left_p:
        target = (left_p + right_p) / 2
        base = BASE_CUTS[5]
        delta = CLAMP_DELTA[5]
        cuts[5] = max(base - delta, min(base + delta, target))
    if left_p is not None and right_p is None:
        cuts[5] = max(cuts[5], left_p + 1)

    # Annual Revenue↔Annual NAFI boundary (cuts[6])
    left_p = pct(annual_rev_right, 100)
    right_p = pct(annual_nafi_left, 0)
    if left_p is not None and right_p is not None and right_p > left_p:
        target = (left_p + right_p) / 2
        base = BASE_CUTS[6]
        delta = CLAMP_DELTA[6]
        cuts[6] = max(base - delta, min(base + delta, target))
    if left_p is not None and right_p is None:
        cuts[6] = max(cuts[6], left_p + 1)

    return cuts

# ---------------- Row extraction ----------------
def rows_from_page(page, cuts, hy, x_tol, y_tol, edge_pad, gap_ratio, drop_min):
    words = chars_to_words(page, x_tol=x_tol, y_tol=y_tol)
    body = [w for w in words if w["top"] > hy + 1 and not _is_exclude_token(w)]

    rows, cur_y, cur = [], None, []
    for w in sorted(body, key=lambda w: (round(w["top"], 1), w["x0"])):
        y = round(w["top"], 1)
        if cur_y is None or abs(y - cur_y) <= y_tol:
            cur.append(w)
            cur_y = y if cur_y is None else (cur_y + y) / 2
        else:
            rows.append(cur)
            cur = [w]
            cur_y = y
    if cur:
        rows.append(cur)

    out = []
    for r in rows:
        buckets = {i: [] for i in range(len(cuts) - 1)}
        for w in r:
            xmid = (w["x0"] + w["x1"]) / 2
            bi = assign_bin(xmid, cuts, edge_pad)
            if bi is not None:
                buckets[bi].append(w)

        vals = [smart_join(buckets[i], x_tol, gap_ratio) for i in range(len(cuts) - 1)]

        # If the third column (Month) looks like money and the fourth
        # column (Revenue) looks like a month, swap them
        if len(vals) >= 4:
            m, rv = vals[2].strip(), vals[3].strip()
            if (
                m and NUM_RE_STRICT.match(m)
                and rv and (_is_month_token(rv))
            ):
                vals[2], vals[3] = rv, m

        if sum(1 for v in vals if v not in ("", "-")) >= drop_min:
            out.append(vals)

    return pd.DataFrame(out, columns=COLS)

# ---------------- Repair helpers ----------------
def repair_loc_and_location(df):
    mask = df["Loc #"].astype(str).str.match(r"^\d+\s+\S+", na=False)
    if mask.any():
        ex = df.loc[mask, "Loc #"].astype(str).str.extract(r"^(\d+)\s+(.*)$")
        df.loc[mask, "Loc #"] = ex[0]
        df.loc[mask, "Location"] = (
            ex[1].fillna("").str.strip() + " " + df.loc[mask, "Location"].fillna("")
        ).str.strip().replace({"": None})

    # Append pure 6-digit codes onto the previous row's Location
    for i in range(len(df) - 1, 0, -1):
        cur_loc = str(df.at[i, "Location"]).strip()
        if re.fullmatch(r"\d{6}", cur_loc):
            prev_loc = str(df.at[i - 1, "Location"]).strip()
            if prev_loc:
                df.at[i - 1, "Location"] = (prev_loc + " " + cur_loc).strip()
                df.at[i, "Location"] = np.nan

    for i in range(1, len(df)):
        cur_locnum = str(df.at[i, "Loc #"]) if pd.notna(df.at[i, "Loc #"]) else ""
        cur_loc = str(df.at[i, "Location"]) if pd.notna(df.at[i, "Location"]) else ""
        prev_loc = (
            str(df.at[i - 1, "Location"]) if pd.notna(df.at[i - 1, "Location"]) else ""
        )
        if re.fullmatch(r"\d{6}", cur_locnum) and (cur_loc == "" or cur_loc.lower() == "nan") and prev_loc not in ("", "nan"):
            df.at[i - 1, "Location"] = (prev_loc + " " + cur_locnum).strip()
            df.at[i, "Loc #"] = df.at[i - 1, "Loc #"]

    return df

def split_and_swap_month_revenue(df):
    def split_cell(mon, rev):
        ms = None if pd.isna(mon) else str(mon).strip()
        rs = None if pd.isna(rev) else str(rev).strip()
        if ms:
            cleaned = ms.replace(",", "")
            toks = re.split(r"\s+", cleaned)
            mon_tok = next((t for t in toks if _is_month_token(t)), None)
            money_toks = [t for t in toks if not _is_month_token(t)]
            money = "".join(money_toks)
            if mon_tok:
                ms = mon_tok
            if money and not (rs and NUM_RE_STRICT.match(rs)):
                rs = money
        if rs and _is_month_token(rs):
            if ms and (NUM_RE_STRICT.match(ms) or NUM_RE_STRICT.match(ms.replace("$", ""))):
                ms, rs = rs, ms
        ms = ms if (ms and ms.strip("-")) else None
        rs = rs if (rs and rs.strip("-")) else None
        return ms, rs

    tmp = df.apply(lambda r: split_cell(r["Month"], r["Revenue"]), axis=1, result_type="expand")
    df["Month"], df["Revenue"] = tmp[0], tmp[1]
    return df

def normalize_months(df):
    def norm(m):
        if pd.isna(m):
            return m
        s = str(m).strip()
        if MONTH_REV.match(s):
            parts = re.split(r'[-/]', s)
            if len(parts) == 2:
                year, mon = parts
                mon = mon[:3].capitalize()
                return mon + '-' + year
        return s

    df["Month"] = df["Month"].apply(norm)
    return df

def _seeded_ffill(series: pd.Series, seed):
    s = series.replace({"": None}).copy()
    if s.empty:
        return s
    if pd.isna(s.iloc[0]) and seed is not None:
        s.iloc[0] = seed
    return s.ffill()

def pagewise_seeded_ffill(dfp: pd.DataFrame, prev_locnum, prev_loc):
    for col in ["Loc #", "Location"]:
        dfp[col] = dfp[col].replace({"": None})
    dfp["Loc #"] = _seeded_ffill(dfp["Loc #"], prev_locnum)
    dfp["Location"] = _seeded_ffill(dfp["Location"], prev_loc)
    return dfp

def finalize(df):
    mask = df["Month"].astype(str).str.match(MONTH_ONLY, na=False) & df["Revenue"].astype(str).isin({str(x) for x in YEAR_FRAG})
    df.loc[mask, "Month"] = df.loc[mask, "Month"] + "-" + df.loc[mask, "Revenue"].astype(str).str[-2:]
    df.loc[mask, "Revenue"] = np.nan

    df["Loc #"] = df["Loc #"].replace({"": None}).ffill()
    df["Location"] = df["Location"].replace({"": None}).ffill()

    for c in ["Revenue", "NAFI Amt", "Annual Revenue", "Annual NAFI"]:
        df[c] = (
            df[c].astype(str)
            .str.replace(" ", "", regex=False)
            .str.replace(",", "", regex=False)
            .str.replace("$", "", regex=False)
            .str.replace("(", "-", regex=False)
            .str.replace(")", "", regex=False)
        )
        df[c] = pd.to_numeric(df[c], errors="coerce")

    return df.dropna(how="all").reset_index(drop=True)

def monthly_numeric_left_pack(df):
    num_cols = ["Revenue", "NAFI Amt", "Annual Revenue", "Annual NAFI"]

    def is_monthlike(x):
        if pd.isna(x):
            return False
        s = str(x).strip()
        return bool(MONTH_FULL.match(s) or MONTH_REV.match(s) or MONTH_ONLY.match(s))

    def pack_row(r):
        if not is_monthlike(r.get("Month")):
            return r
        vals = [r[c] for c in num_cols]
        avail = [v for v in vals if pd.notna(v)]
        if avail:
            r["Revenue"] = avail[0]
            r["NAFI Amt"] = avail[1] if len(avail) > 1 else np.nan
            r["Annual Revenue"] = avail[2] if len(avail) > 2 else np.nan
            r["Annual NAFI"] = avail[3] if len(avail) > 3 else np.nan
        return r

    return df.apply(pack_row, axis=1)

def fill_missing_annual(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    month_map = {
        m: i
        for i, m in enumerate(
            ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"], start=1
        )
    }

    def parse_date(m):
        if pd.isna(m):
            return None
        m = str(m)
        if re.match(r"^[A-Za-z]{3}-\d{2}$", m):
            mon, yr = m.split('-')
            return (2000 + int(yr), month_map.get(mon[:3], 0))
        return None

    df["_ym"] = df["Month"].apply(parse_date)

    for (locnum, loc), idxs in df.groupby(["Loc #", "Location"], dropna=False).groups.items():
        idxs = list(idxs)
        sub = df.loc[idxs].copy()
        sub["_ym_key"] = sub["_ym"].apply(lambda t: t[0] * 100 + t[1] if t else None)
        sub = sub.sort_values("_ym_key")

        revenues = sub["Revenue"].fillna(0).to_list()
        nafis = sub["NAFI Amt"].fillna(0).to_list()

        for i, (idx, row) in enumerate(sub.iterrows()):
            ym = row["_ym"]
            if (
                (pd.isna(row["Annual Revenue"]) or pd.isna(row["Annual NAFI"]))
                and ym
                and ym[1] == 9
            ):
                start = max(0, i - 11)
                s_rev = sum(revenues[start : i + 1])
                s_naf = sum(nafis[start : i + 1])
                if pd.isna(row["Annual Revenue"]):
                    df.at[idx, "Annual Revenue"] = s_rev
                if pd.isna(row["Annual NAFI"]):
                    df.at[idx, "Annual NAFI"] = s_naf

    df.drop(columns=["_ym"], inplace=True, errors="ignore")
    df.drop(columns=["_ym_key"], inplace=True, errors="ignore")
    return df

# ---- Default base cut lines and allowed clamp deltas ----
BASE_CUTS = [-1e9, 156.21, 214.20, 269.19, 327.93, 393.67, 460.17, 1e9]
CLAMP_DELTA = [0, 30, 40, 40, 30, 30, 30, 0]

def apply_page_params(page_num):
    """Determine parameter overrides for a given page.
    Returns a dictionary of parameters to use when processing ``page_num``.
    Values not specified in an override fall back to the base values.
    """
    params = {
        "Y_TOL": BASE_Y_TOL,
        "X_JOIN": BASE_X_JOIN,
        "GAP_RATIO": BASE_GAP_RATIO,
        "DROP_MIN": BASE_DROP_MIN,
        "EDGE_PAD": BASE_EDGE_PAD,
        "LEFT_SHIFT_MONTH": BASE_LEFT_SHIFT_MONTH,
        "LEFT_SHIFT_REVENUE": BASE_LEFT_SHIFT_REVENUE,
    }
    # find any matching override range
    for rng, overrides in SPECIAL_PARAMS.items():
        if page_num in rng:
            params.update(overrides)
            break
    return params

# ---------------- Main processing routine ----------------
def main():
    all_pages = []
    prev_loc = None
    prev_locnum = None
    prev_cuts = None

    with pdfplumber.open(PDF) as pdf:
        last = min(202, len(pdf.pages))

        # reference header y from page 2 (index 1) if needed
        words2 = chars_to_words(pdf.pages[1], x_tol=BASE_X_JOIN, y_tol=BASE_Y_TOL)
        header2_y = header_y_of(words2)

        # skip pages used for slot revenue tables
        skip_pages = {1, 35, 73, 115, 157}

        # iterate pages starting from 2
        for page_num in range(2, last + 1):
            if page_num in skip_pages:
                print(f"[p{page_num}] SKIPPED.")
                continue

            # Determine parameter set for this page
            params = apply_page_params(page_num)
            y_tol = params["Y_TOL"]
            x_tol = params["X_JOIN"]
            gap_ratio = params["GAP_RATIO"]
            drop_min = params["DROP_MIN"]
            edge_pad = params["EDGE_PAD"]
            left_shift_month = params["LEFT_SHIFT_MONTH"]
            left_shift_revenue = params["LEFT_SHIFT_REVENUE"]

            page = pdf.pages[page_num - 1]
            words = chars_to_words(page, x_tol=x_tol, y_tol=y_tol)

            # derive cuts from header; if no header found use previous or base
            cuts, hy = cuts_from_header(
                words,
                BASE_CUTS,
                prev_cuts=prev_cuts,
                left_shift_month=left_shift_month,
                left_shift_revenue=left_shift_revenue,
            )
            hy = hy or header2_y

            # refine cuts to account for numbers crossing boundaries
            cuts = refine_cuts_typeaware(page, cuts, hy)

            # split page into rows
            dfp = rows_from_page(
                page,
                cuts,
                hy,
                x_tol=x_tol,
                y_tol=y_tol,
                edge_pad=edge_pad,
                gap_ratio=gap_ratio,
                drop_min=drop_min,
            )

            if dfp.empty:
                print(f"[p{page_num}] rows=0")
                continue

            # repair fields and normalize months
            dfp = repair_loc_and_location(dfp)
            dfp = split_and_swap_month_revenue(dfp)
            dfp = normalize_months(dfp)
            dfp = pagewise_seeded_ffill(dfp, prev_locnum, prev_loc)
            dfp = finalize(dfp)
            dfp = monthly_numeric_left_pack(dfp)

            # update seeds for cross-page continuity
            if dfp["Location"].notna().any():
                prev_loc = dfp["Location"].dropna().iloc[-1]
            if dfp["Loc #"].notna().any():
                prev_locnum = dfp["Loc #"].dropna().iloc[-1]
            prev_cuts = cuts

            # annotate with original page number
            dfp.insert(0, "Page", page_num)
            all_pages.append(dfp)
            print(f"[p{page_num}] rows={len(dfp)}")

    if all_pages:
        out = pd.concat(all_pages, ignore_index=True)
        # Do not automatically fill annual totals; respect original numbers
        out.to_csv(OUT, index=False)
        print(f"✅ Done. rows={len(out)} → {OUT}")
    else:
        print("No data")

# 1) Set the PDF path (if the file is under /content)
PDF = Path('/content/Marine_Revenue_FY20-FY24.pdf')
OUT = PDF.parent / 'Marine_Revenue_FY20-FY24_detail.csv'

# 2) Override global variables, then run main()
globals()['PDF'] = PDF
globals()['OUT'] = OUT
print('PDF =', PDF)
print('OUT =', OUT)
main()


PDF = /content/Marine_Revenue_FY20-FY24.pdf
OUT = /content/Marine_Revenue_FY20-FY24_detail.csv
[p2] rows=14
[p3] rows=38
[p4] rows=20
[p5] rows=41
[p6] rows=43
[p7] rows=29
[p8] rows=41
[p9] rows=43
[p10] rows=8
[p11] rows=40
[p12] rows=42
[p13] rows=47
[p14] rows=39
[p15] rows=44
[p16] rows=41
[p17] rows=44
[p18] rows=41
[p19] rows=44
[p20] rows=38
[p21] rows=40
[p22] rows=44
[p23] rows=33
[p24] rows=39
[p25] rows=4
[p26] rows=26
[p27] rows=41
[p28] rows=42
[p29] rows=39
[p30] rows=17
[p31] rows=39
[p32] rows=38
[p33] rows=36
[p34] rows=10
[p35] SKIPPED.
[p36] rows=14
[p37] rows=40
[p38] rows=30
[p39] rows=36
[p40] rows=38
[p41] rows=37
[p42] rows=15
[p43] rows=44
[p44] rows=47
[p45] rows=21
[p46] rows=38
[p47] rows=42
[p48] rows=41
[p49] rows=44
[p50] rows=25
[p51] rows=44
[p52] rows=43
[p53] rows=42
[p54] rows=44
[p55] rows=40
[p56] rows=44
[p57] rows=40
[p58] rows=40
[p59] rows=44
[p60] rows=32
[p61] rows=41
[p62] rows=16
[p63] rows=26
[p64] rows=40
[p65] rows=44
[p66] rows=11
[p67